In [ ]:
# hide

import blackhc.project.script
from nbdev.showdoc import *

Appended /home/blackhc/PycharmProjects/bald-ical/src to paths
Switched to directory /home/blackhc/PycharmProjects/bald-ical
%load_ext autoreload
%autoreload 2


# Extension: Stochastic Acquisition
> Greedy algorithm and score computation

First, we will implement two helper classes to compute conditional entropies $H[y_i|w]$ and entropies $H[y_i]$. 
Then, we will implement BatchBALD and BALD.

In [ ]:
import math

import numpy as np
import torch
from blackhc.progress_bar import create_progress_bar
from toma import toma

from batchbald_redux.acquisition_functions.stochastic_acquisition import * 
from batchbald_redux.joint_entropy import *

We are going to define a couple of sampled distributions to use for our testing our code.

$K=20$ means 20 inference samples.

In [ ]:
K = 20

In [ ]:
def get_mixture_prob_dist(p1, p2, m):
    return (1.0 - m) * np.asarray(p1) + m * np.asarray(p2)


p1 = [0.7, 0.1, 0.1, 0.1]
p2 = [0.3, 0.3, 0.2, 0.2]
y1_ws = [get_mixture_prob_dist(p1, p2, m) for m in np.linspace(0, 1, K)]

p1 = [0.1, 0.7, 0.1, 0.1]
p2 = [0.2, 0.3, 0.3, 0.2]
y2_ws = [get_mixture_prob_dist(p1, p2, m) for m in np.linspace(0, 1, K)]

p1 = [0.1, 0.1, 0.7, 0.1]
p2 = [0.2, 0.2, 0.3, 0.3]
y3_ws = [get_mixture_prob_dist(p1, p2, m) for m in np.linspace(0, 1, K)]

p1 = [0.1, 0.1, 0.1, 0.7]
p2 = [0.3, 0.2, 0.2, 0.3]
y4_ws = [get_mixture_prob_dist(p1, p2, m) for m in np.linspace(0, 1, K)]


def nested_to_tensor(l):
    return torch.stack(list(map(torch.as_tensor, l)))


ys_ws = nested_to_tensor([y1_ws, y2_ws, y3_ws, y4_ws])

In [ ]:
# hide

p = [0.25, 0.25, 0.25, 0.25]
yu_ws = [p for m in range(K)]
yus_ws = nested_to_tensor([yu_ws] * 4)

In [ ]:
ys_ws.shape

torch.Size([4, 20, 4])

However, our neural networks usually use a `log_softmax` as final layer. To avoid having to call `.exp_()`, which is easy to miss and annoying to debug, we will instead use a version that uses `log_probs` instead of `probs`.

In [ ]:
# hide

# Make sure everything is computed correctly.
assert np.allclose(compute_conditional_entropy(yus_ws.log()), [1.3863, 1.3863, 1.3863, 1.3863], atol=0.1)
assert np.allclose(compute_entropy(yus_ws.log()), [1.3863, 1.3863, 1.3863, 1.3863], atol=0.1)

Conditional Entropy:   0%|          | 0/4 [00:00<?, ?it/s]

Entropy:   0%|          | 0/4 [00:00<?, ?it/s]

### Examples

In [ ]:
conditional_entropies = compute_conditional_entropy(ys_ws.log())

print(conditional_entropies)

assert np.allclose(conditional_entropies, [1.2069, 1.2069, 1.2069, 1.2069], atol=0.01)

Conditional Entropy:   0%|          | 0/4 [00:00<?, ?it/s]

tensor([1.2069, 1.2069, 1.2069, 1.2069], dtype=torch.float64)


In [ ]:
entropies = compute_entropy(ys_ws.log())

print(entropies)

assert np.allclose(entropies, [1.2376, 1.2376, 1.2376, 1.2376], atol=0.01)

Entropy:   0%|          | 0/4 [00:00<?, ?it/s]

tensor([1.2376, 1.2376, 1.2376, 1.2376], dtype=torch.float64)


## Stochastic Acquisition

Re-implementation for the final paper experiments (hopefully without bugs...)

In [ ]:
assert get_power_samples(torch.as_tensor([1, 0, 0]), coldness=1, batch_size=1).indices == [0]

In [ ]:
np.unique(
    sum(
        (get_power_samples(torch.as_tensor([0.5, 0.25, 0.25]), coldness=1, batch_size=1).indices for _ in range(1000)),
        [],
    ),
    return_counts=True,
)[1] / 1000
# should be around [0.5, 0.25, 0.25]

array([0.516, 0.235, 0.249])

In [ ]:
np.unique(
    sum(
        (
            get_softmax_samples(torch.as_tensor([np.log(2), 0, 0]), coldness=1, batch_size=1).indices
            for _ in range(1000)
        ),
        [],
    ),
    return_counts=True,
)[1] / 1000
# should be around [0.5, 0.25, 0.25]

array([0.486, 0.261, 0.253])

In [ ]:
np.unique(
    sum((get_softrank_samples(torch.as_tensor([3, 2, 1]), coldness=1, batch_size=1).indices for _ in range(1000)), []),
    return_counts=True,
)[1] / 1000 * 11
# p = [1, 1/2, 1/3] / ((6+3+2)/6) = [1, 1/2, 1/3] * 6 / 11 = [6/11, 3/11, 2/11]

array([5.973, 3.212, 1.815])

In [ ]:
np.unique(
    sum((get_softrank_samples(torch.as_tensor([3, 2, 1]), coldness=8, batch_size=1).indices for _ in range(1000)), []),
    return_counts=True,
)[1] / 1000 * 11

array([10.989,  0.011])

In [ ]:
np.unique(
    sum((get_softmax_samples(torch.as_tensor([3, 2, 1]), coldness=0, batch_size=1).indices for _ in range(1000)), []),
    return_counts=True,
)[1] / 1000

array([0.387, 0.303, 0.31 ])

In [ ]:
np.unique(
    sum((get_softmax_samples(torch.as_tensor([3, 2, 1]), coldness=0, batch_size=1).indices for _ in range(1000)), []),
    return_counts=True,
)[1] / 1000

array([0.334, 0.306, 0.36 ])

In [ ]:
[get_stochastic_samples(torch.randn(size=(4,)).exp_(), coldness=1, batch_size=4, mode=mode) for mode in StochasticMode]

[CandidateBatch(scores=[0.6314359795733706, 0.39256970788054596, -0.9629585341370686, -1.3475644155039097], indices=[1, 2, 0, 3]),
 CandidateBatch(scores=[5.016618910618873, 4.678579359150341, 1.1375936366639063, 0.3820472708228937], indices=[1, 2, 0, 3]),
 CandidateBatch(scores=[0.29905812048994007, -1.2632030660251827, -1.3416275883912618, -1.4855583100638685], indices=[2, 3, 1, 0]),
 CandidateBatch(scores=[3.7639763355255127, 1.7821424026392407, 0.5014993000216927, 0.010073202340172595], indices=[0, 3, 2, 1]),
 CandidateBatch(scores=[5.81153678894043, 3.05381178855896, 0.37783998250961304, 0.3310677409172058], indices=[1, 0, 3, 2])]